In [43]:
import pandas as pd 
import numpy as np 
import sys  
sys.path.insert(0, 'C/Users/lero1/Desktop/songML')
import math
from random import randrange
import random
import string
import category_encoders as ce 
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [44]:
# drop rows with invalid values and destringify the list of artists 
def cleanArtists(songs):
    songs['artists'] = songs['artists'].apply(lambda x: x[1:-1].split(', ') if(type(x) == str and len(x)) else [])
    songs['artists'] = songs['artists'].apply(lambda x: list(map(lambda y: y[1:-1], x)) )
    encoder = ce.HashingEncoder(cols=['artists'], n_components=200)
    encodedSongs = encoder.fit_transform(songs['artists'], songs['popularity'])
    df = pd.DataFrame(encodedSongs,index=songs.index)
    songs = df.join(songs)
    return songs

In [45]:
# drop rows with invalid values or out of range (797 rows)
def cleanYear(songs):
     songs = songs.dropna(subset=['year'])
     songs.loc[songs.year > 1900, 'year'] = 2020 - songs.year
     songs = songs.rename(columns={'year': 'yearsSinceCreation'})
     return songs

In [46]:
# merging groups of songs with the same name and artists taking mean values for the other columns
def mergeDuplicates(songs):
    songs = songs.groupby(['artists', 'name'], as_index=False).agg({
        'valence':np.average,
        'yearsSinceCreation':np.average,
        'acousticness':np.average,
        'danceability':np.average,
        'duration_ms':np.average,
        'energy':np.average,
        'instrumentalness':np.average,
        'liveness':np.average,
        'loudness':np.average,
        'tempo':np.average,
        'speechiness':np.average,
        'explicit':np.average,
        'mode':np.average,
        'popularity':np.average
    })
    songs.loc[:, 'explicit'] = round(songs.explicit)

    return songs

In [47]:
def removeEmpty(songs):
    for col in songs.columns:
        songs = songs.dropna(subset=[col])
    return songs

In [48]:
def dropCols(songs):
    songs.drop(columns=['id', 'release_date'])
    return songs

In [49]:
songs = pd.read_csv('spotify_training.csv')
songs = cleanYear(songs)
songs = removeEmpty(songs)
songs = dropCols(songs)
print(len(songs))
# songs = mergeDuplicates(songs)
for col in songs.columns:
    print(col, '\n')


118631
valence 

yearsSinceCreation 

acousticness 

artists 

danceability 

duration_ms 

energy 

explicit 

id 

instrumentalness 

key 

liveness 

loudness 

mode 

name 

tempo 

release_date 

speechiness 

popularity 



In [50]:
songs = cleanArtists(songs)
songs.describe()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,energy,explicit,instrumentalness,key,liveness,loudness,mode,tempo,speechiness,popularity
count,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,...,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000
mean,0.007216,0.006881,0.004808,0.007362,0.004163,0.005204,0.004326,0.003200,0.006674,0.011706,...,0.508376,0.078091,0.158576,5.198582,0.205758,-11.066926,0.706131,125.939342,0.093396,32.309927
std,0.084642,0.082665,0.069173,0.085489,0.064386,0.071949,0.065633,0.056475,0.081424,0.107560,...,0.266269,0.268316,0.305123,3.521117,0.175990,5.475479,0.455535,24.836022,0.154357,21.620667
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.284000,0.000000,0.000000,2.000000,0.097900,-14.043000,0.000000,109.857000,0.035000,14.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.506000,0.000000,0.000217,5.000000,0.135000,-10.202000,1.000000,124.116000,0.045000,34.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.730000,0.000000,0.083200,8.000000,0.262000,-6.964000,1.000000,140.395000,0.073200,49.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,11.000000,1.000000,3.744000,1.000000,181.368000,0.970000,100.000000


In [51]:
songs = songs.drop(columns=['name', 'artists'])
songs.describe()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,energy,explicit,instrumentalness,key,liveness,loudness,mode,tempo,speechiness,popularity
count,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,...,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000,118631.000000
mean,0.007216,0.006881,0.004808,0.007362,0.004163,0.005204,0.004326,0.003200,0.006674,0.011706,...,0.508376,0.078091,0.158576,5.198582,0.205758,-11.066926,0.706131,125.939342,0.093396,32.309927
std,0.084642,0.082665,0.069173,0.085489,0.064386,0.071949,0.065633,0.056475,0.081424,0.107560,...,0.266269,0.268316,0.305123,3.521117,0.175990,5.475479,0.455535,24.836022,0.154357,21.620667
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.284000,0.000000,0.000000,2.000000,0.097900,-14.043000,0.000000,109.857000,0.035000,14.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.506000,0.000000,0.000217,5.000000,0.135000,-10.202000,1.000000,124.116000,0.045000,34.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.730000,0.000000,0.083200,8.000000,0.262000,-6.964000,1.000000,140.395000,0.073200,49.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,11.000000,1.000000,3.744000,1.000000,181.368000,0.970000,100.000000


In [52]:
songs.dropna(how='any',inplace=True)
songs.describe()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,energy,explicit,instrumentalness,key,liveness,loudness,mode,tempo,speechiness,popularity
count,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,...,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.000000,116265.00000,116265.000000,116265.000000
mean,0.007216,0.006881,0.004808,0.007362,0.004163,0.005204,0.004326,0.003200,0.006674,0.011706,...,0.513246,0.065901,0.160902,5.202512,0.204714,-10.965918,0.707092,125.96125,0.082229,32.808257
std,0.084642,0.082665,0.069173,0.085489,0.064386,0.071949,0.065633,0.056475,0.081424,0.107560,...,0.265565,0.248110,0.306791,3.515102,0.176139,5.440611,0.455099,24.78719,0.121022,21.415504
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.00000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.292000,0.000000,0.000000,2.000000,0.097600,-13.863000,0.000000,109.88900,0.034900,16.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.514000,0.000000,0.000249,5.000000,0.134000,-10.109000,1.000000,124.12300,0.044700,35.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.734000,0.000000,0.090600,8.000000,0.259000,-6.911000,1.000000,140.42400,0.071000,49.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,11.000000,1.000000,3.744000,1.000000,181.36800,0.968000,100.000000


In [53]:
from reg import reg
print("top featrues")
reg(songs, True)
print("all featrues")
reg(songs, False)

top featrues
Index(['top freatures :yearsSinceCreation', 'top freatures :acousticness',
       'top freatures :energy', 'top freatures :loudness',
       'top freatures :popularity'],
      dtype='object')


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').